# Euclid Q1 Lenses

* **Phil Marshall, Phil Holloway, Ralf Kaehler, Ferro Shao**
* DP1
* data.lsst.cloud
* r29.1.1
* Fri July 11 2025

## Goals

* Extract _ugrizy_ coadd image cutouts for each Euclid Q1 strong lens candidate in the ECDFS and EDFS DP1 fields
* Visualize them as _gri_ color composites.
* Stretch: deconvolve them using the Rubin SharPy by Kaehler et al (in prep)

In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afw_display
import lsst.geom as geom
import numpy as np
import matplotlib.pyplot as plt

afw_display.setDefaultBackend('matplotlib')

## Cutout Image Extraction

First we need to make a list (or better, a table) of targets. Then, for each one, we find out which DP1 coadd patch it lies in. (We'll need to choose which patch, for systems that lie in the patch overlap regions and hence in multiple patches.) Then, we loop over patches and bands, uploading a patch image and extracting all the cutouts we can - which will mean getting the image coordinates for each system

In [ ]:
butler = Butler("dp1", collections="LSSTComCam/DP1")
assert butler is not None

In [ ]:
butler.get_dataset_type('deep_coadd').dimensions.required

## Single Sky Position Testing

### Single Band

Let's just try extracting a single 32x32 pixel cutout image in one band.

In [ ]:
ra = 59.626134
dec = -49.06175

band = 'i'

We need to find the tract and patch that this target is in:

In [ ]:
radec = geom.SpherePoint(ra, dec, geom.degrees)
cutoutSize = geom.ExtentI(32, 32)

skymap = butler.get("skyMap")
tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)

patch = tractInfo.getSequentialPatchIndex(patchInfo)
tract = tractInfo.getId()

dataId = {'tract': tract, 'patch': patch, 'band': band}

When testing, it can be useful to upload the whole patch image and inspect it.

In [ ]:
# deep_coadd = butler.get('deep_coadd', band=band, tract=tract, patch=patch)
# coadd

In [ ]:
# fig = plt.figure(figsize=(6,6))
# display = afw_display.Display(frame=fig)
# display.scale('asinh', 'zscale')
# display.mtv(coadd.image)
# plt.show()

Now to define a small bounding box, and extract the pixels in it. This first cell _should_ work, but doesn't - maybe some tract/patch confusion. There could be some speed up here at some point, making multiple cutouts from teh same patch image.

In [ ]:
# xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
# bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

# cutout = coadd.Factory(coadd, bbox)

Here's some code that does work: define the bounding box, then just grab that part of the image.

In [ ]:
xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

parameters = {'bbox': bbox}

cutout = butler.get("deep_coadd", parameters=parameters, dataId=dataId)

Quick look to check we got our object!

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
plt.show()

## Multiple Bands

Loop over all 6 bands:

In [ ]:
bands = ["u","g","r","i","z","y"]
cutout = {}

for band in bands:
    dataId = {'tract': tract, 'patch': patch, 'band': band}
    cutout[band] = butler.get("deep_coadd", parameters=parameters, dataId=dataId)

In [ ]:
cutout

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout["y"].image)
plt.show()

OK - we have 6 cutouts for this target, so can go ahead and make a color composite. It took about 5 secs to make them all: we'll need to keep an eye on this, and return to the `factory` approach to try and speed things up a bit.

## _gri_ Composite Image Visualization

## Do-It-All Code

## Appendix

The code below is from the Cutout Factory demo notebook by Melissa Graham, and is being used as a source in this notebook further up.

In [ ]:


import lsst.afw.display as afw_display
from lsst.daf.butler import Butler
import lsst.geom as geom
import matplotlib.pyplot as plt

afw_display.setDefaultBackend('matplotlib')

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
dataId = {'visit': 192350, 'detector': 175, 'band': 'i'}
calexp = butler.get('calexp', **dataId)

In [ ]:
fig = plt.figure(figsize=(3,3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(calexp.image)
plt.show()

In [ ]:
cutoutSize = geom.ExtentI(301, 301)

xy1 = geom.PointI(2250, 700)
bbox1 = geom.BoxI(xy1 - cutoutSize // 2, cutoutSize)

xy2 = geom.PointI(400, 1750)
bbox2 = geom.BoxI(xy2 - cutoutSize // 2, cutoutSize)

In [ ]:
cutout1 = calexp.Factory(calexp, bbox1)
cutout2 = calexp.Factory(calexp, bbox2)

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout1.image)
plt.show()

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout2.image)
plt.show()